In [1]:
import torch
import torch.nn as nn


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

In [4]:
Data = pd.DataFrame(data = mnist['data'], columns = mnist['feature_names'])

In [5]:
Target = pd.DataFrame(data = mnist['target'], columns = ['class'])

In [6]:
# Data['Target'] = Target

In [7]:
Train_Data = Data[:60000]
Test_Data = Data[60001:]

# Train_Data['kfold'] = -1

In [61]:
Target_train = Target[:60000]
Target_test = Target[60001:]

# Kfold

In [47]:
# def kfold(df,splits):
#     df = df.sample(frac =1).reset_index(drop = True)
#     # Define a Stratified K fold model 
#     kf = StratifiedKFold(n_splits = splits, shuffle=False, random_state=42)

#     # Loop over all the fold indexes
#     for fold, (train_idx, val_idx) in enumerate(kf.split(X = df, y = df.Target.values)):
#         print(len(train_idx), len(val_idx), fold)
#         # print(fold)
#         # Change the "kfold" column values to their respetive fold numbers
#         df.loc[val_idx, "kfold"] = fold
#     # return the strafiedkfold file
#     return df

In [67]:
# Train_Data = kfold(Train_Data,5)

In [ ]:
# fold = [0,1,2,3,4]
# for f in fold:
#     train_df = Train_Data[]

# def train_test_split(Data, Target, Test_Size, Random_State):
#     xtrain, xvalid, ytrain, yvalid = train_test_split(Data, Target, test_size = Test_Size, random_state = Random_State)
#     return xtrain, xvalid, ytrain, yvalid

In [112]:
def normalize(x):
    # '0' for get max values columnwise:
    x_normed = x / x.max(0, keepdim=True)[0]
    return x_normed

In [9]:
Train_Data = np.array(Train_Data)
Test_Data = np.array(Test_Data)
Train_Data = Train_Data.astype(dtype = 'float64')
Test_Data = Test_Data.astype(dtype = 'float64')
Train_Data = torch.from_numpy(Train_Data)
Test_Data = torch.from_numpy(Test_Data)
# Train_Data_norm = normalize(Train_Data)
# Test_Data_norm = normalize(Test_Data) 
# Train_Data_norm[torch.isnan(Train_Data_norm)] = 0
# Test_Data_norm[torch.isnan(Test_Data_norm)] = 0

In [62]:
Target_train = np.array(Target_train)
Target_test = np.array(Target_test)
Target_train = Target_train.astype(int)
Target_test = Target_test.astype(int)
Target_train = torch.from_numpy(Target_train)
Target_test = torch.from_numpy(Target_test)

In [35]:
# enc = OneHotEncoder()
# Target_test = enc.fit_transform(Target_test)
# Target_train = enc.fit_transform(Target_train)
Target_test = torch.nn.functional.one_hot(Target_test)
Target_train = torch.nn.functional.one_hot(Target_train)


In [36]:
Target_train.resize_(60000,10)
Target_test.resize_(9999,10)

tensor([[0, 0, 1,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [90]:
Test_Data.size()

torch.Size([9999, 784])

In [63]:
X = Train_Data
y = Target_train
xPredicted = Test_Data

In [93]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Initilalize parameters:
        self.inputsize = 784
        self.outputsize = 10
        # self.outputsize = 1
        self.hiddensize = 255

        #Weights
        self.W1 = torch.randn(self.inputsize, self.hiddensize, dtype = torch.float64)
        self.W2 = torch.randn(self.hiddensize, self.outputsize, dtype = torch.float64)

        # self.W2 = torch.randn(self.hiddensize, self.hiddensize, dtype = torch.float64)
        # self.W3 = torch.randn(self.hiddensize, self.outputsize, dtype = torch.float64)


    def forward(self, X):
        self.z = torch.matmul(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = torch.matmul(self.z2, self.W2)
        # self.z4 = self.sigmoid(self.z3)
        # self.z5 = torch.matmul(self.z4, self.W3)
        # o = self.relu(self.z5)
        # print(self.z5.size())
        o = self.sigmoid(self.z3)
        return o

    # def relu(self, x):
    #     z = x.max(1, keepdim = True)[0]
    #     return torch.tensor(z, dtype = torch.float64)

    # def reluprime(self, x):
    #     x[x>0] = 1
    #     x[x<0] = 0
    #     return x

    
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))

    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s *(1-s)
        # return self.sigmoid(s) * (1 - self.sigmoid(s))
    
    def backward(self, X, y, o):
        # print(o.size(),y.size())
        # print(o,y)
        self.o_error = y - o
        # print(self.o_error)
        # print(self.sigmoidPrime(o))
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        # print(self.o_delta)
        # self.o_delta = self.o_error * self.reluprime(o)
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        # print(self.z2_error)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        # print(self.z2_delta)
        # self.z3_error = torch.matmul(self.z2_delta, torch.t(self.W2))
        # print(self.z3_error)
        # self.z3_delta = self.z3_error * self.sigmoidPrime(self.z2)
        # print(self.z3_delta)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        # self.W3 += torch.matmul(torch.t(self.z3), self.o_delta)
        print("weights")
        print(self.W1)
        # self.W1 += torch.matmul(torch.t(X), self.z3_delta)
        # self.W2 += torch.matmul(torch.t(self.z2), self.z2_delta)
        # self.W3 += torch.matmul(torch.t(self.z3), self.o_delta)

    def train(self,X,y):
        o = self.forward(X)
        self.backward(X,y,o)
    
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")

    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        output = self.forward(xPredicted)
        print ("Output: \n" + str(output))
        return output


In [94]:
NN = NeuralNetwork()
for i in range(3):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
# y_pred = NN.predict()

#0 Loss: 24.236135693039063
weights
tensor([[-0.3250, -0.5902, -0.0390,  ..., -0.7386,  0.5673, -0.0562],
        [ 2.2707,  0.7966, -0.8493,  ...,  0.5128, -0.7668,  0.0459],
        [-0.7067, -0.5483, -0.1746,  ..., -0.1492, -0.7241, -0.2063],
        ...,
        [-0.1483,  0.9439,  0.8487,  ..., -0.3792,  0.2696,  1.0657],
        [-0.3258, -0.2393,  0.2355,  ...,  0.0541,  0.9762, -0.5477],
        [ 0.5661, -0.5319,  1.2399,  ..., -0.3002,  1.3770,  0.4254]],
       dtype=torch.float64)
#1 Loss: 20.2774
weights
tensor([[-0.3250, -0.5902, -0.0390,  ..., -0.7386,  0.5673, -0.0562],
        [ 2.2707,  0.7966, -0.8493,  ...,  0.5128, -0.7668,  0.0459],
        [-0.7067, -0.5483, -0.1746,  ..., -0.1492, -0.7241, -0.2063],
        ...,
        [-0.1483,  0.9439,  0.8487,  ..., -0.3792,  0.2696,  1.0657],
        [-0.3258, -0.2393,  0.2355,  ...,  0.0541,  0.9762, -0.5477],
        [ 0.5661, -0.5319,  1.2399,  ..., -0.3002,  1.3770,  0.4254]],
       dtype=torch.float64)
#2 Loss: 20.277

In [55]:
torch.max(y_pred,1)[1]

tensor([9, 9, 9,  ..., 9, 9, 9])

In [60]:
y_pred[9]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)